In [1]:
#import bibliotek
import json
import requests
import pandas as pd
import time
from time import sleep
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

Dokumentacja API: [klik](https://api-datalab.coderslab.com/v2/docs/).

#definicja tokenu i adresu url w celu połączenia z API
api_key = "iKRsQ8vdqgT903o2vH1rsejOeQ0F7YC9TvutH6Wk"
api_url = "https://api-datalab.coderslab.com/api/v2"
auth = {'authorization':api_key}

#wczytanie pliku
airports_df = pd.read_csv(r"../data/airports.csv")

# Dostosowanie listy lotnisk do dalszych kroków
origin_airport_ids = airports_df['origin_airport_id'].tolist()



#Pobieranie danych z endpoint'u airport

# Lista do przechowywania danych o lotniskach
airport_data_list = []

# Pętla do pobierania danych dla każdego lotniska
for airport_id in origin_airport_ids:
    url = f"{api_url}/airport/{airport_id}"
    response = requests.get(url, headers= auth)
    if response.status_code == 200:
        airport_data_list.append(response.json())
    else:
        print(f"Error downloading data for airport id {airport_id}")
    time.sleep(0.5)

#Zapisywanie danych do tabeli
airport_df = pd.DataFrame.from_records(airport_data_list)

#Zapisanie ramki do pliku
airport_df.to_csv(r"../data/raw/airport_list.csv", index=False)



#Pobieranie danych z endpoint'u weather

endpoint = "/airportWeather"

# Ustawienie daty początkowej i końcowej
start_date = datetime(2019, 1, 1)
end_date = datetime(2020, 3, 31)

# Lista, która będzie przechowywać dane
weather_data = []

current_date = start_date
# Pętla przez okres czasu
while current_date <= end_date:
    # Formatowanie daty
    formatted_date = current_date.strftime("%Y-%m-%d")
    
    # Tworzenie URL-a z odpowiednią datą
    url = f"{api_url}{endpoint}?date={formatted_date}"
    
    # Wykonanie zapytania HTTP
    response = requests.get(url, headers={'authorization': api_key})
    
    # Sprawdzenie poprawności odpowiedzi
    if response.status_code == 200:
        data = response.json()
        weather_data.extend(data)
        print(f"The data has been downloaded for {formatted_date}")
    else:
        print(f"Error downloading data for {formatted_date}")
    
    # Przejście do kolejnego miesiąca
    current_date += relativedelta(months=1)
    sleep(0.5)
    
#Zapisywanie danych do tabeli
airport_weather_df = pd.DataFrame(weather_data)

#Zapisanie ramki do pliku
airport_weather_df.to_csv(r"../data/raw/airport_weather.csv", index=False)



#Pobieranie danych z endpoint'u aircraft
endpoint = "/aircraft"

# Lista, która będzie przechowywać dane
aircraft_data = []

# Tworzenie URL-a 
url = f"{api_url}{endpoint}"
    
# Wykonanie zapytania HTTP
response = requests.get(url, headers={'authorization': api_key})
    
# Sprawdzenie poprawności odpowiedzi
if response.status_code == 200:
    data = response.json()
    # Zachowanie danych w liście
    aircraft_data.extend(data)
    print("The data has been downloaded and saved to the list")
else:
    print("Error downloading data")
sleep(0.5)

#Zapisywanie danych do tabeli
aircraft_df = pd.DataFrame(aircraft_data)

#Zapisanie ramki do pliku
aircraft_df.to_csv(r"../data/raw/aircraft.csv", index=False)



#Pobieranie danych z endpoint'u flight
endpoint_ft = "/flight"
# Lista do przechowywania danych o pogodzie na lotniskach
flight_data_list = []

# Data początkowa
start_date = datetime(2019, 1, 1)
current_date = start_date

# Pobranie danych dla każdego lotniska
for airport_id in origin_airport_ids:
    
    # Pętla przez okres czasu
    current_date = start_date
    for i in range(15):
        # Formatowanie daty
        formatted_date = current_date.strftime("%Y-%m")
    
        # Tworzenie URL-a
        url = f"{api_url}{endpoint_ft}?date={formatted_date}&airportId={airport_id}"
        
        # Wykonanie zapytania HTTP
        response = requests.get(url, headers={'authorization': api_key})
        
        # Sprawdzenie poprawności odpowiedzi
        if response.status_code == 200:
            data = response.json()
            if data:  # Sprawdzenie, czy dane istnieją
                flight_data_list.extend(data)
                print(f"Data downloaded for airport {airport_id} on {formatted_date}")
            else:
                print(f"No data available for the airport {airport_id} on {formatted_date}")
        elif response.status_code == 204:
            print(f"No data available for the airport {airport_id} on {formatted_date}")
            break
        # Przejście do kolejnej daty
        current_date += relativedelta(months=1)

#Zapisywanie danych do tabeli
flight_df = pd.DataFrame(flight_data_list)



#Zapisanie ramki do pliku
flight_df.to_csv(r"../data/raw/flight_df.csv", index=False)